# Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd /content/drive/MyDrive/netFound/src/train/

In [ ]:
!pip install datasets

In [ ]:
!pip install torchinfo

In [ ]:
!pip install tensorboard

In [ ]:
import warnings
from sklearn.exceptions import UndefinedMetricWarning
warnings.filterwarnings("ignore", category=UndefinedMetricWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

import os
import torch
import torch.distributed
import numpy as np
import utils
import random
from dataclasses import field, dataclass
from datasets.distributed import split_dataset_by_node
from typing import Optional
from copy import deepcopy
from torchinfo import summary
from torch.distributed.elastic.multiprocessing.errors import record

from transformers import (
    EvalPrediction,
    HfArgumentParser,
    TrainingArguments,
    EarlyStoppingCallback,
)

from sklearn.metrics import (
    f1_score,
    accuracy_score,
    precision_score,
    recall_score,
    top_k_accuracy_score,
    classification_report, confusion_matrix
)

from NetFoundDataCollator import DataCollatorForFlowClassification
from NetFoundModels import NetfoundFinetuningModel, NetfoundNoPTM
from NetFoundTrainer import NetfoundTrainer
from NetfoundConfig import NetfoundConfig, NetFoundTCPOptionsConfig, NetFoundLarge
from NetfoundTokenizer import NetFoundTokenizer

from utils import ModelArguments, CommonDataTrainingArguments, freeze, verify_checkpoint, \
    load_train_test_datasets, get_90_percent_cpu_count, get_logger, init_tbwriter, update_deepspeed_config, \
    LearningRateLogCallback

random.seed(42)
logger = get_logger(name=__name__)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
import torch.optim as optim
import torch.nn as nn
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

# Funtions

In [ ]:
@dataclass
class FineTuningDataTrainingArguments(CommonDataTrainingArguments):
    """
    Arguments pertaining to what data we are going to input our model for training and eval.
    """

    num_labels: int = field(metadata={"help": "number of classes in the datasets"}, default=None)
    problem_type: Optional[str] = field(
        default=None,
        metadata={"help": "Override regression or classification task"},
    )
    p_val: float = field(
        default=0,
        metadata={
            "help": "noise rate"
        },
    )
    netfound_large: bool = field(
        default=False,
        metadata={
            "help": "Use the large configuration for netFound model"
        },
    )

In [ ]:
def regression_metrics(p: EvalPrediction):
    logits = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    label_ids = p.label_ids.astype(int)
    return {"loss": np.mean(np.absolute((logits - label_ids)))}

In [ ]:
def classif_metrics(p: EvalPrediction, num_classes):
    logits = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    label_ids = p.label_ids.astype(int)
    weighted_f1 = f1_score(
        y_true=label_ids, y_pred=logits.argmax(axis=1), average="weighted", zero_division=0
    )
    weighted_prec = precision_score(
        y_true=label_ids, y_pred=logits.argmax(axis=1), average="weighted", zero_division=0
    )
    weighted_recall = recall_score(
        y_true=label_ids, y_pred=logits.argmax(axis=1), average="weighted", zero_division=0
    )
    accuracy = accuracy_score(y_true=label_ids, y_pred=logits.argmax(axis=1))
    logger.warning(classification_report(label_ids, logits.argmax(axis=1), digits=5))
    logger.warning(confusion_matrix(label_ids, logits.argmax(axis=1)))
    if num_classes > 3:
        logger.warning(f"top3:{top_k_accuracy_score(label_ids, logits, k=3, labels=np.arange(num_classes))}")
    if num_classes > 5:
        logger.warning(f"top5:{top_k_accuracy_score(label_ids, logits, k=5, labels=np.arange(num_classes))}")
    if num_classes > 10:
        logger.warning(f"top10:{top_k_accuracy_score(label_ids, logits, k=10, labels=np.arange(num_classes))}")
    return {
        "weighted_f1": weighted_f1,
        "accuracy": accuracy,
        "weighted_prec: ": weighted_prec,
        "weighted_recall": weighted_recall,
    }

# Load Model and Dataset

In [ ]:
dataset_folder = r"/content/drive/MyDrive/netFound/data/test/test_finetuning/final/combined/"
pretrained_model_path = r"/content/drive/MyDrive/netFound/models/test/original_pretraining/pretrained_model/"
output_dir = r"/content/drive/MyDrive/netFound/models/test/finetuning/"

In [ ]:
parser = HfArgumentParser((ModelArguments, FineTuningDataTrainingArguments, TrainingArguments))
model_args, data_args, training_args = parser.parse_args_into_dataclasses(args=["--train_dir", dataset_folder, "--test_dir", dataset_folder, "--model_name_or_path", pretrained_model_path, "--output_dir", output_dir, "--report_to", "tensorboard", "--overwrite_output_dir", "--save_safetensors", "false", "--do_train", "--do_eval", "--eval_strategy", "epoch", "--save_strategy", "epoch", "--learning_rate", "0.01", "--num_train_epochs", "1", "--problem_type", "single_label_classification", "--num_labels", "6", "--load_best_model_at_end", "--netfound_large", "True"])
# utils.LOGGING_LEVEL = training_args.get_process_log_level()
utils.LOGGING_LEVEL = 10
logger.setLevel(10)

In [ ]:
logger.info(f"model_args: {model_args}")
logger.info(f"data_args: {data_args}")
logger.info(f"training_args: {training_args}")

In [ ]:
train_dataset, test_dataset = load_train_test_datasets(logger, data_args)

In [ ]:
len(train_dataset), len(test_dataset)

In [ ]:
config = NetFoundTCPOptionsConfig if data_args.tcpoptions else NetfoundConfig

In [ ]:
config = config(num_hidden_layers=model_args.num_hidden_layers, num_attention_heads=model_args.num_attention_heads,
        hidden_size=model_args.hidden_size, no_meta=data_args.no_meta, flat=data_args.flat)

In [ ]:
if data_args.netfound_large:
    config.hidden_size = NetFoundLarge().hidden_size
    config.num_hidden_layers = NetFoundLarge().num_hidden_layers
    config.num_attention_heads = NetFoundLarge().num_attention_heads

In [ ]:
config.pretraining = False
config.num_labels = data_args.num_labels
config.problem_type = data_args.problem_type
testingTokenizer = NetFoundTokenizer(config=config)

In [ ]:
training_config = deepcopy(config)
training_config.p = data_args.p_val
training_config.limit_bursts = data_args.limit_bursts
trainingTokenizer = NetFoundTokenizer(config=training_config)
additionalFields = None

In [ ]:
params = {"batched": True}

In [ ]:
train_dataset = train_dataset.map(function=trainingTokenizer, **params)
test_dataset = test_dataset.map(function=testingTokenizer, **params)

In [ ]:
data_collator = DataCollatorForFlowClassification(config.max_burst_length)

In [ ]:
logger.warning(f"Using weights from {model_args.model_name_or_path}")

In [ ]:
original_model = NetfoundFinetuningModel.from_pretrained(model_args.model_name_or_path, config=config)

In [ ]:
model = freeze(original_model, model_args)

# Infer Model

In [ ]:
saved_weights = os.path.join(output_dir, f'model_epoch_latest.pth')
if os.path.exists(saved_weights):
    weights = torch.load(saved_weights)
    model.load_state_dict(weights)

In [ ]:
for x in train_dataset:
    break

In [ ]:
px = data_collator([x])

In [ ]:
py = model(labels = px['labels'], protocol = px['protocol'], flow_duration = px['flow_duration'], bytes = px['bytes'], iats = px['iats'], input_ids = px['input_ids'], attention_mask = px['attention_mask'], direction = px['direction'], pkt_count = px['pkt_count'])

In [ ]:
py.logits

# Train Model

In [ ]:
model = model.to(device)
model = model.train()

In [ ]:
req_keys = ['labels', 'protocol', 'flow_duration', 'bytes', 'iats', 'input_ids', 'attention_mask', 'direction', 'pkt_count', 'total_bursts']

In [ ]:
for p in model.base_transformer.parameters():
    p.requires_grad = False
for p in model.attentivePooling.parameters():
    p.requires_grad = False
for p in model.dropout.parameters():
    p.requires_grad = True
for p in model.hiddenLayer.parameters():
    p.requires_grad = True
for p in model.hiddenLayer2.parameters():
    p.requires_grad = True
for p in model.classifier.parameters():
    p.requires_grad = True

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)
batch_size = 64

In [ ]:
train_len = len(train_dataset)
train_inds = np.arange(train_len)
np.random.shuffle(train_inds)
train_inds = train_inds.tolist()

In [ ]:
xl = []
for i, ind in enumerate(train_inds):
  x = train_dataset[ind]
  if i%batch_size != batch_size - 1:
    xx = {k: x[k] for k in x if k in req_keys}
    xl.append(xx)
    continue

  px = data_collator(xl)
  print(px['labels'])
  xl = []

In [ ]:
ground_truths = []
predictions = []
xl = []
with torch.no_grad():
    for i, x in enumerate(test_dataset, 0):
      try:
        if i%batch_size != batch_size - 1:
          xx = {k: x[k] for k in x if k in req_keys}
          xl.append(xx)
          continue

        px = data_collator(xl)
        px = {a: b.to(device) for a,b in px.items()}

        py = model(labels = px['labels'], protocol = px['protocol'], flow_duration = px['flow_duration'], bytes = px['bytes'], iats = px['iats'],
                   input_ids = px['input_ids'], attention_mask = px['attention_mask'], direction = px['direction'], pkt_count = px['pkt_count'])
        labels = px['labels']
        outputs = py.logits
        _, predicted = torch.max(outputs, 1)
        ground_truths += labels.cpu().numpy().tolist()
        predictions += predicted.cpu().numpy().tolist()
        xl = []
      except Exception as e:
        print(i, e)
        pass

accuracy = accuracy_score(ground_truths, predictions)
print('Accuracy: %.3f' % accuracy)

precision = precision_score(ground_truths, predictions, average='weighted')
print('Precision: %.3f' % precision)

recall = recall_score(ground_truths, predictions, average='weighted')
print('Recall: %.3f' % recall)

In [ ]:
print(classification_report(ground_truths, predictions, target_names=['0', '1', '2', '3', '4', '5']))

In [ ]:
prev_accuracy = accuracy

In [ ]:
for epoch in range(1, 50):
    running_loss = 0.0
    running_count = 0
    xl = []
    for i, ind in enumerate(train_inds, 0):
      x = train_dataset[ind]
      try:
        if i%batch_size != batch_size - 1:
          xx = {k: x[k] for k in x if k in req_keys}
          xl.append(xx)
          continue

        px = data_collator(xl)
        px = {a: b.to(device) for a,b in px.items()}
        optimizer.zero_grad()

        py = model(labels = px['labels'], protocol = px['protocol'], flow_duration = px['flow_duration'], bytes = px['bytes'], iats = px['iats'],
                   input_ids = px['input_ids'], attention_mask = px['attention_mask'], direction = px['direction'], pkt_count = px['pkt_count'])
        loss = criterion(py.logits, px['labels'])
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        running_count += 1
        xl = []
      except Exception as e:
        print(i, e)
        break
    print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / running_count:.3f} loss count: {running_count}')

    ground_truths = []
    predictions = []
    xl = []
    with torch.no_grad():
        for i, x in enumerate(test_dataset, 0):
          try:
            if i%batch_size != batch_size - 1:
              xx = {k: x[k] for k in x if k in req_keys}
              xl.append(xx)
              continue

            px = data_collator(xl)
            px = {a: b.to(device) for a,b in px.items()}

            py = model(labels = px['labels'], protocol = px['protocol'], flow_duration = px['flow_duration'], bytes = px['bytes'], iats = px['iats'],
                      input_ids = px['input_ids'], attention_mask = px['attention_mask'], direction = px['direction'], pkt_count = px['pkt_count'])
            labels = px['labels']
            outputs = py.logits
            _, predicted = torch.max(outputs, 1)
            ground_truths += labels.cpu().numpy().tolist()
            predictions += predicted.cpu().numpy().tolist()
            xl = []
          except Exception as e:
            print(i, e)
            pass

    accuracy = accuracy_score(ground_truths, predictions)
    print('Epoch: %d, Accuracy: %.3f' % (epoch, accuracy))

    precision = precision_score(ground_truths, predictions, average='weighted')
    print('Epoch: %d, Precision: %.3f' % (epoch, precision))

    recall = recall_score(ground_truths, predictions, average='weighted')
    print('Epoch: %d, Recall: %.3f' % (epoch, recall))

    print("Epoch: %d"%epoch)
    print(classification_report(ground_truths, predictions, target_names=['0', '1', '2', '3', '4', '5']))

    if prev_accuracy < accuracy:
      prev_accuracy = accuracy
      torch.save(model.state_dict(), os.path.join(output_dir, f'model_epoch_latest.pth'))
      print("Stored:", epoch)
print('Finished Training')

In [ ]:
from google.colab import runtime
runtime.unassign()